In [5]:
import os
import torch
import torchvision
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data_dir  = 'c:/Users/Reliance Digital/Downloads/Images'

classes = os.listdir(data_dir)
print(classes)

['airport_inside', 'artstudio', 'auditorium', 'bakery', 'bar', 'bathroom', 'bedroom', 'bookstore', 'bowling', 'buffet', 'casino', 'children_room', 'church_inside', 'classroom', 'cloister', 'closet', 'clothingstore', 'computerroom', 'concert_hall', 'corridor', 'deli', 'dentaloffice', 'dining_room', 'elevator', 'fastfood_restaurant', 'florist', 'gameroom', 'garage', 'greenhouse', 'grocerystore', 'gym', 'hairsalon', 'hospitalroom', 'inside_bus', 'inside_subway', 'jewelleryshop', 'kindergarden', 'kitchen', 'laboratorywet', 'laundromat', 'library', 'livingroom', 'lobby', 'locker_room', 'mall', 'meeting_room', 'movietheater', 'museum', 'nursery', 'office', 'operating_room', 'pantry', 'poolinside', 'prisoncell', 'restaurant', 'restaurant_kitchen', 'shoeshop', 'stairscase', 'studiomusic', 'subway', 'toystore', 'trainstation', 'tv_studio', 'videostore', 'waitingroom', 'warehouse', 'winecellar']


In [4]:
len(classes)

67

In [6]:
home_interior_classes = [
    'livingroom', 'bedroom', 'kitchen', 'bathroom', 'dining_room', 
    'closet', 'pantry', 'children_room', 'nursery', 'home_office'
]

In [7]:
import shutil

In [8]:
filtered_dir = './home_interior_dataset'
os.makedirs(filtered_dir, exist_ok=True)

In [9]:
# Create subdirectories for each home interior class
for class_name in home_interior_classes:
    src_dir = os.path.join(data_dir, class_name)
    dst_dir = os.path.join(filtered_dir, class_name)
    if os.path.exists(src_dir):
        shutil.copytree(src_dir, dst_dir)


In [10]:
# Transformations
transformations = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor()
])

In [11]:
# Load filtered dataset
dataset = ImageFolder(filtered_dir, transform=transformations)

In [12]:
# Helper function to show sample
def show_sample(img, label):
    print("Label:", dataset.classes[label], "(Class No: "+ str(label) + ")")
    plt.imshow(img.permute(1, 2, 0))

In [13]:
# Split dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [14]:
# Data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [15]:
# Model definition
class IndoorSceneClassifier(nn.Module):
    def __init__(self, num_classes=len(home_interior_classes)):
        super(IndoorSceneClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
    
    def forward(self, x):
        return self.model(x)

In [16]:
# Initialize model, loss function, and optimizer
model = IndoorSceneClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

C:\Users\Reliance Digital\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Reliance Digital\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Reliance Digital/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 55.7MB/s]


In [17]:
# Training function
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        

In [18]:
        # Validation
model.eval()
val_loss = 0.0
val_corrects = 0
        
with torch.no_grad():
    for inputs, labels in val_loader:
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                
    val_loss = val_loss / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)
        
    print(f'Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

Validation Loss: 2.5022 Acc: 0.0881


In [ ]:
import os
import torch
import torchvision
from torch.utils.data import random_split
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
%matplotlib inline

# Define home interior classes
home_interior_classes = [
    'livingroom', 'bedroom', 'kitchen', 'bathroom', 'dining_room', 
    'closet', 'pantry', 'children_room', 'nursery', 'home_office'
]

# Original data directory
data_dir = 'c:/Users/Reliance Digital/Downloads/Images'


# Transformations
transformations = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor()
])

# Load filtered dataset
dataset = ImageFolder(filtered_dir, transform=transformations)

# Helper function to show sample
def show_sample(img, label):
    print("Label:", dataset.classes[label], "(Class No: "+ str(label) + ")")
    plt.imshow(img.permute(1, 2, 0))

# Split dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Data loaders
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Model definition
class IndoorSceneClassifier(nn.Module):
    def __init__(self, num_classes=len(home_interior_classes)):
        super(IndoorSceneClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
    
    def forward(self, x):
        return self.model(x)

# Initialize model, loss function, and optimizer
model = IndoorSceneClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            
        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.double() / len(train_dataset)
        
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                
        val_loss = val_loss / len(val_dataset)
        val_acc = val_corrects.double() / len(val_dataset)
        
        print(f'Validation Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

# Train the model
train_model(model, criterion, optimizer, num_epochs=10)

# Save the model
torch.save(model.state_dict(), 'home_interior_classifier.pth')

# Function to predict on new images
def predict_image(image_path):
    image = Image.open(image_path)
    image = transformations(image).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output.data, 1)
        
    return dataset.classes[predicted.item()]

# Example usage
# predicted_class = predict_image('path_to_your_image.jpg')
# print(f'Predicted class: {predicted_class}')

Epoch 1/10 - Loss: 1.0942 Acc: 0.6303
Validation Loss: 1.4639 Acc: 0.5373
Epoch 2/10 - Loss: 0.6805 Acc: 0.7651
Validation Loss: 2.1709 Acc: 0.4179
Epoch 3/10 - Loss: 0.4319 Acc: 0.8514
Validation Loss: 1.1955 Acc: 0.6000
Epoch 4/10 - Loss: 0.3813 Acc: 0.8708


In [ ]:
# Save the model
torch.save(model.state_dict(), 'home_interior_classifier.pth')

In [1]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageDraw
import cv2
import albumentations as A
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.ops import box_iou
import torch.nn.functional as F
from ultralytics import YOLO

# =============================================
# Configuration
# =============================================
class Config:
    # Dataset
    data_dir = "home_interior_dataset"  # Update this
    classes = [
        'livingroom', 'bedroom', 'kitchen', 'bathroom', 'dining_room',
        'closet', 'pantry', 'children_room', 'nursery'
    ]
    furniture_classes = [
        '__background__', 'bed', 'chair', 'sofa', 'table', 'desk',
        'cabinet', 'shelf', 'refrigerator', 'oven', 'toilet'
    ]
    
    # Image parameters
    cls_img_size = 384
    det_img_size = 512
    batch_size = 16
    
    # Training
    num_epochs = 30
    lr = 1e-4
    weight_decay = 1e-4
    
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = Config()

# =============================================
# ETL Pipeline
# =============================================
def create_annotation_structure():
    """Generate directory structure and annotation template"""
    os.makedirs(os.path.join(config.data_dir, "annotations"), exist_ok=True)
    
    structure = {}
    for cls in config.classes:
        cls_dir = os.path.join(config.data_dir, cls)
        if os.path.exists(cls_dir):
            images = [f for f in os.listdir(cls_dir) if f.endswith(('.jpg', '.png'))]
            for img in images:
                img_id = f"{cls}_{img}"
                img_path = os.path.join(cls_dir, img)
                with Image.open(img_path) as im:
                    width, height = im.size
                
                structure[img_id] = {
                    "file_path": img_path,
                    "width": width,
                    "height": height,
                    "class": cls,
                    "boxes": [],
                    "labels": []
                }
    
    # Save template
    with open(os.path.join(config.data_dir, "annotations/template.json"), 'w') as f:
        json.dump(structure, f)
    print("Annotation template created. Please manually annotate or use synthetic data.")

# =============================================
# EDA
# =============================================
def perform_eda():
    """Exploratory Data Analysis"""
    class_counts = {}
    image_sizes = []
    
    for cls in config.classes:
        cls_dir = os.path.join(config.data_dir, cls)
        if os.path.exists(cls_dir):
            images = os.listdir(cls_dir)
            class_counts[cls] = len(images)
            
            # Sample image size
            img = random.choice(images)
            with Image.open(os.path.join(cls_dir, img)) as im:
                image_sizes.append(im.size)
    
    # Plot class distribution
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
    plt.title('Class Distribution')
    plt.xticks(rotation=45)
    
    # Plot image sizes
    plt.subplot(1, 2, 2)
    widths, heights = zip(*image_sizes)
    sns.scatterplot(x=widths, y=heights, alpha=0.5)
    plt.title('Image Size Distribution')
    plt.tight_layout()
    plt.show()
    
    print("\nDataset Statistics:")
    print(f"Total classes: {len(class_counts)}")
    print(f"Total images: {sum(class_counts.values())}")
    print(f"Average per class: {sum(class_counts.values())/len(class_counts):.1f}\n")
    print("Class counts:", class_counts)

# =============================================
# Data Preprocessing
# =============================================
class RoomDataset(Dataset):
    """Classification Dataset"""
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.class_to_idx = {cls: i for i, cls in enumerate(config.classes)}
        self.images = self._load_images()
        
    def _load_images(self):
        images = []
        for cls in config.classes:
            cls_dir = os.path.join(self.root, cls)
            if os.path.exists(cls_dir):
                for img in os.listdir(cls_dir):
                    if img.endswith(('.jpg', '.png')):
                        images.append((os.path.join(cls_dir, img), self.class_to_idx[cls]))
        return images
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path, label = self.images[idx]
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
            
        return img, label

def get_classification_transforms():
    """Data augmentation for classification"""
    train_transform = transforms.Compose([
        transforms.Resize((config.cls_img_size, config.cls_img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize((config.cls_img_size, config.cls_img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    return train_transform, val_transform

# =============================================
# Model Training (Classification)
# =============================================
def train_classification_model():
    """Train room classifier"""
    train_transform, val_transform = get_classification_transforms()
    
    # Load datasets
    full_dataset = RoomDataset(config.data_dir, transform=train_transform)
    
    # Split dataset
    train_idx, val_idx = train_test_split(
        list(range(len(full_dataset))),
        test_size=0.2,
        stratify=[label for _, label in full_dataset.images]
    )
    
    train_dataset = torch.utils.data.Subset(full_dataset, train_idx)
    val_dataset = torch.utils.data.Subset(full_dataset, val_idx)
    val_dataset.dataset.transform = val_transform
    
    # Data loaders
    train_loader = DataLoader(
        train_dataset, batch_size=config.batch_size, shuffle=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=config.batch_size, shuffle=False
    )
    
    # Model
    model = models.efficientnet_b3(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(config.classes))
    model = model.to(config.device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=config.lr,
        weight_decay=config.weight_decay
    )
    
    # Training loop
    best_acc = 0.0
    for epoch in range(config.num_epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(config.device), labels.to(config.device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(config.device), labels.to(config.device)
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
                _, preds = torch.max(outputs, 1)
                correct += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(train_loader)
        val_loss = val_loss / len(val_loader)
        val_acc = correct.double() / len(val_dataset)
        
        print(f"Epoch {epoch+1}/{config.num_epochs}")
        print(f"Train Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
        
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_classifier.pth")
    
    print(f"Best Val Acc: {best_acc:.4f}")

# =============================================
# Object Detection (YOLOv8)
# =============================================
def prepare_detection_data():
    """Convert annotations to YOLO format"""
    os.makedirs("yolo_data", exist_ok=True)
    os.makedirs("yolo_data/images/train", exist_ok=True)
    os.makedirs("yolo_data/labels/train", exist_ok=True)
    
    # Load annotations (replace with your annotation file)
    with open(os.path.join(config.data_dir, "annotations.json")) as f:
        annotations = json.load(f)
    
    for img_id, ann in annotations.items():
        # Copy image
        img = Image.open(ann["file_path"])
        img.save(f"yolo_data/images/train/{img_id}.jpg")
        
        # Create YOLO format label file
        label_lines = []
        for box, label_idx in zip(ann["boxes"], ann["labels"]):
            x1, y1, x2, y2 = box
            width = ann["width"]
            height = ann["height"]
            
            # Convert to YOLO format (normalized cx, cy, w, h)
            cx = ((x1 + x2) / 2) / width
            cy = ((y1 + y2) / 2) / height
            w = (x2 - x1) / width
            h = (y2 - y1) / height
            
            label_lines.append(f"{label_idx} {cx} {cy} {w} {h}\n")
        
        with open(f"yolo_data/labels/train/{img_id}.txt", 'w') as f:
            f.writelines(label_lines)
    
    print("YOLO dataset prepared")

def train_detection_model():
    """Train YOLOv8 detector"""
    # Create YOLO config file
    yolo_config = {
        "path": "yolo_data",
        "train": "images/train",
        "val": "images/train",  # For demo, use same as train
        "names": {i: name for i, name in enumerate(config.furniture_classes)}
    }
    
    with open("yolo_data/furniture.yaml", 'w') as f:
        yaml.dump(yolo_config, f)
    
    # Load and train model
    model = YOLO("yolov8n.pt")
    model.train(
        data="yolo_data/furniture.yaml",
        epochs=50,
        imgsz=config.det_img_size,
        batch=config.batch_size
    )
    
    # Save best model
    model.export(format="onnx")

# =============================================
# Main Execution
# =============================================
if __name__ == "__main__":
    # Step 1: EDA
    print("Performing EDA...")
    perform_eda()
    
    # Step 2: Classification Training
    print("\nTraining Classification Model...")
    train_classification_model()
    
    # Step 3: Detection Training
    print("\nPreparing Detection Data...")
    prepare_detection_data()
    print("Training Detection Model...")
    train_detection_model()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Reliance Digital\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\Reliance Digital\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\Reliance Digital\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", 

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = ImageFolder(data_dir, transform = transformations)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_sample(img, label):
    print("Label:", dataset.classes[label], "(Class No: "+ str(label) + ")")
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
img, label = dataset[50]
show_sample(img, label)

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)


In [ ]:
len(dataset)

In [ ]:
train_ds, val_ds, test_ds = random_split(dataset, [13000, 2000, 620])
len(train_ds), len(val_ds), len(test_ds)

In [ ]:
from torch.utils.data.dataloader import DataLoader
batch_size = 25

In [ ]:
train_dl = DataLoader(train_ds, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers = 4, pin_memory = True)

In [ ]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images, nrow = 16).permute(1, 2, 0))
        break

In [ ]:
show_batch(train_dl)


In [ ]:
show_batch(val_dl)